# BIOPS QS JSON Option Use Case 1: Move one dashboard to another account with assets as bundle API #
Author: Ying Wang, Sr. SA in Gen AI, QuickSight

Created Time: June 2025

**Only run it one time!!!**

The cell below is to update the bucket policy to allow cross-accounts access: let the targe account to copy the assets in source account.
Please update the template path if you have a new assets bundle to be imported into the target account

In [ ]:
import boto3
import json
from botocore.exceptions import ClientError

# === Configuration ===
bucket_name = "biops-version-control-demo-2025"
source_account_id = ""  # Replace with the AWS Account ID to allow
target_account_id = ""
#template_path = ""  # S3 object key (or prefix)

# === Initialize S3 client ===
s3 = boto3.client("s3")

# === Define the new policy statement ===
new_statement1 = {
    "Sid": "AllowAccountAToAccessTemplate",
    "Effect": "Allow",
    "Principal": {
        "AWS": f"arn:aws:iam::{target_account_id}:root"
    },
    "Action": "s3:*",
      "Resource": [
        f"arn:aws:s3:::{bucket_name}",
        f"arn:aws:s3:::{bucket_name}/*"
      ]
}

new_statement2 = {
      "Sid": "AllowCloudFormationAccess",
      "Effect": "Allow",
      "Principal": {
        "AWS": f"arn:aws:iam::{target_account_id}:role/QuickSightAdminConsole2025"
      },
      "Action": "s3:GetObject",
      "Resource": f"arn:aws:s3:::{bucket_name}/*"
    }


# Try to get the existing policy, or create a new one
try:
    response = s3.get_bucket_policy(Bucket=bucket_name)
    policy = json.loads(response['Policy'])
except s3.exceptions.NoSuchBucketPolicy:
    policy = {
        "Version": "2012-10-17",
        "Statement": []
    }

# Always append the new statement (no duplicate check)
policy["Statement"].append(new_statement1)
policy["Statement"].append(new_statement2)

# Update the policy
updated_policy = json.dumps(policy)
s3.put_bucket_policy(Bucket=bucket_name, Policy=updated_policy)
print("Bucket policy updated successfully.")

## Start source account boto3 session ##

In [ ]:
import json
import boto3
import logging
import csv
import io
import os
import tempfile
from typing import Any, Callable, Dict, List, Optional, Union
import sys
import botocore

#start-Initial set up for the sdk env#
def default_botocore_config() -> botocore.config.Config:
    """Botocore configuration."""
    retries_config: Dict[str, Union[str, int]] = {
        "max_attempts": int(os.getenv("AWS_MAX_ATTEMPTS", "5")),
    }
    mode: Optional[str] = os.getenv("AWS_RETRY_MODE")
    if mode:
        retries_config["mode"] = mode
    return botocore.config.Config(
        retries=retries_config,
        connect_timeout=10,
        max_pool_connections=10,
        user_agent_extra=f"qs_sdk_biops",
    )
sts_client = boto3.client("sts", config=default_botocore_config())
account_id = sts_client.get_caller_identity()["Account"]
#qs_client = boto3.client('quicksight', region_name=aws_region, config=default_botocore_config())
qs_client = boto3.client('quicksight')
aws_region = qs_client.meta.region_name
print(aws_region)

In [ ]:
permissions=qs_client.describe_analysis_definition(
    AwsAccountId=account_id,
    AnalysisId='c5769da4-3e7b-421c-ab74-e8c891eccea3',
    
)
from IPython.display import JSON
JSON(permissions)

In [ ]:
import json
import boto3
import logging
import csv
import io
import os
import tempfile
from typing import Any, Callable, Dict, List, Optional, Union
import sys
import botocore
from datetime import datetime

#start-Initial set up for the sdk env#
def default_botocore_config() -> botocore.config.Config:
    """Botocore configuration."""
    retries_config: Dict[str, Union[str, int]] = {
        "max_attempts": int(os.getenv("AWS_MAX_ATTEMPTS", "5")),
    }
    mode: Optional[str] = os.getenv("AWS_RETRY_MODE")
    if mode:
        retries_config["mode"] = mode
    return botocore.config.Config(
        retries=retries_config,
        connect_timeout=10,
        max_pool_connections=10,
        user_agent_extra=f"qs_sdk_biops",
    )
sts_client = boto3.client("sts", config=default_botocore_config())
account_id = sts_client.get_caller_identity()["Account"]
aws_region = 'us-east-1'
#qs_client = boto3.client('quicksight', region_name=aws_region, config=default_botocore_config())
qs_client = boto3.client('quicksight')


"""
permissions=qs_client.describe_analysis(
    AwsAccountId=account_id,
    AnalysisId='',
    
)
"""



# Get current local time
current_time = datetime.now()

dashboard_id = ''
jobId = dashboard_id + '_' + account_id + '_' + str(current_time)
response = qs_client.start_asset_bundle_export_job(
    AwsAccountId=account_id,
    AssetBundleExportJobId=jobId,
    ResourceArns=[
        f"arn:aws:quicksight:us-east-1:{account_id}:dashboard/{dashboard_id}",
    ],
    IncludeAllDependencies=True,
    IncludePermissions=False,
    ExportFormat='QUICKSIGHT_JSON')


from IPython.display import JSON
JSON(response)

In [ ]:

response = qs_client.describe_asset_bundle_export_job(
    AwsAccountId=account_id,
    AssetBundleExportJobId=jobId
)



from IPython.display import JSON
JSON(response)

In [ ]:
import time
def WaitForExportToComplete(jobId):
    while True:
        response = qs_client.describe_asset_bundle_export_job(
                    AwsAccountId=account_id,
                    AssetBundleExportJobId=jobId
                    )
        
        job_status = response['JobStatus']
        if job_status in ['SUCCESSFUL', 'FAILED']:
            print(f"Job finished with status: {job_status}")
            return response
        else:
            print(f"Job still running. Current status: {job_status}. Retrying in 10 seconds...")
            time.sleep(10)

In [ ]:
response=WaitForExportToComplete(jobId)

In [ ]:
import requests
import boto3
import os

# === Configuration ===
url = response['DownloadUrl']
local_path = response['ExportFormat'] + '_' + response['AssetBundleExportJobId']
local_file_name = local_path + '.zip'
bucket_name = "biops-version-control-demo-2025"
s3_key = local_path + '/' + local_file_name

# === Step 1: Download file from URL ===
response = requests.get(url)
if response.status_code == 200:
    with open(local_file_name, "wb") as f:
        f.write(response.content)
else:
    raise Exception(f"Failed to download file. Status code: {response.status_code}")

# === Step 2: Upload file to S3 ===
s3 = boto3.client('s3')
s3.upload_file(local_file_name, bucket_name, s3_key)

print(f"File uploaded to s3://{bucket_name}/{s3_key}")



In [ ]:
s3path = f"{bucket_name}/{s3_key}"
s3url = f"https://{bucket_name}.s3.us-east-1.amazonaws.com/{s3_key}"
s3uri = f"s3://{bucket_name}/{s3_key}"
print(s3uri)

## Start Import job ##

In [ ]:
def _assume_role(aws_account_number, role_name, aws_region):
    sts_client = boto3.client('sts', config=default_botocore_config())
    response = sts_client.assume_role(
        RoleArn='arn:aws:iam::' + aws_account_number + ':role/' + role_name,
        RoleSessionName='quicksight'
    )
    # Storing STS credentials
    session = boto3.Session(
        aws_access_key_id=response['Credentials']['AccessKeyId'],
        aws_secret_access_key=response['Credentials']['SecretAccessKey'],
        aws_session_token=response['Credentials']['SessionToken'],
        region_name=aws_region
    )

    #logger.info("Assumed session for " + aws_account_number + " in region " + aws_region + ".")

    return session

In [ ]:
def get_user_arn (session, username, region='us-east-1', namespace='default'): 
    sts_client = session.client("sts")
    account_id = sts_client.get_caller_identity()["Account"]
    if username=='root':
        arn='arn:aws:iam::'+account_id+':'+username
    else:
        arn="arn:aws:quicksight:"+region+":"+account_id+":user/"+namespace+"/"+username
    
    return arn

In [ ]:

target_account_id = ''

role_name=""
aws_region='us-east-1'
targetsession = _assume_role(target_account_id, role_name, aws_region)


In [ ]:
targetroot=get_user_arn (targetsession, 'root')
targetadmin=get_user_arn (targetsession, 'admin')

In [ ]:
qs_client_target = targetsession.client('quicksight')
sts_client_target = targetsession.client("sts")
target_account_id = sts_client_target.get_caller_identity()["Account"]
print(target_account_id)

In [ ]:
importjobid = dashboard_id + '_' + target_account_id + '_' + str(current_time)
response = qs_client_target.start_asset_bundle_import_job(
    AwsAccountId=target_account_id,
    AssetBundleImportJobId=importjobid,
    AssetBundleImportSource={
        'S3Uri': s3uri
    },
    FailureAction='ROLLBACK'
)

from IPython.display import JSON
JSON(response)

In [ ]:
response = qs_client_target.describe_asset_bundle_import_job(
    AwsAccountId=target_account_id,
    AssetBundleImportJobId=importjobid
)



from IPython.display import JSON
JSON(response)

In [ ]:
import time
def WaitForImportToComplete(jobId):
    while True:
        response = qs_client_target.describe_asset_bundle_import_job(
                    AwsAccountId=target_account_id,
                    AssetBundleImportJobId=importjobid
                    )
        
        job_status = response['JobStatus']
        if job_status in ['SUCCESSFUL', 'FAILED']:
            print(f"Job finished with status: {job_status}")
            return response
        else:
            print(f"Job still running. Current status: {job_status}. Retrying in 10 seconds...")
            time.sleep(10)

In [ ]:
response=WaitForImportToComplete(importjobid)

In [ ]:
response = qs_client_target.search_dashboards(
    AwsAccountId=target_account_id,
    Filters=[
        {
            'Operator': 'StringLike',
            'Name': 'DASHBOARD_NAME',
            'Value': 'BIOpsDemo'
        },
    ]
)

from IPython.display import JSON
JSON(response)

In [ ]:
print(targetadmin)
print(response['DashboardSummaryList'][0]['DashboardId'])
response = qs_client_target.update_dashboard_permissions(
    AwsAccountId=target_account_id,
    DashboardId=response['DashboardSummaryList'][0]['DashboardId'],
    GrantPermissions=[
        {
            'Principal': targetadmin,
            'Actions': [
                "quicksight:DescribeDashboard",
              "quicksight:ListDashboardVersions",
              "quicksight:UpdateDashboardPermissions",
              "quicksight:QueryDashboard",
              "quicksight:UpdateDashboard",
              "quicksight:DeleteDashboard",
              "quicksight:DescribeDashboardPermissions",
              "quicksight:UpdateDashboardPublishedVersion"
            ]
        }
    ]
    
)

from IPython.display import JSON
JSON(response)

In [ ]:
response = qs_client_target.describe_dashboard(
    AwsAccountId=target_account_id,
    DashboardId=''
)

from IPython.display import JSON
JSON(response)

In [ ]:
from IPython.display import JSON

datasets = response['Dashboard']['Version']['DataSetArns']
for dataset in datasets:
    dsid = dataset.split(":")[-1].split("/")[-1]
    res = qs_client_target.update_data_set_permissions(
    AwsAccountId=target_account_id,
    DataSetId = dsid,
    GrantPermissions=[
        {
            'Principal': targetadmin,
            'Actions': [
                 "quicksight:UpdateDataSetPermissions",
              "quicksight:DescribeDataSet",
              "quicksight:DescribeDataSetPermissions",
              "quicksight:PassDataSet",
              "quicksight:DescribeIngestion",
              "quicksight:ListIngestions",
              "quicksight:UpdateDataSet",
              "quicksight:DeleteDataSet",
              "quicksight:CreateIngestion",
              "quicksight:CancelIngestion"
            ]
        }
    ]
    
  )
    JSON(res)
    print(dsid + "permissions updated" )
    
  
    